# Brute force - Code pour plusieurs jours (MAJ suite à sécurité Nitter)

In [71]:
#pip install selenium
#pip install webdriver_manager
#pip install bs4

In [1]:
import datetime
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
import urllib.parse

In [2]:
# Créer une liste pour stocker les résultats de chaque jour
results = []

In [4]:
# Définir la date de début et la date de fin
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 6, 1)

In [5]:
# Chargement du dictionnaire des mots
dict_mot = pd.read_csv('PPD_Dictionnaire_des_mots.csv',sep=';')
print(dict_mot.to_string())

                          Mot  Fréquence
0       transition écologique        339
1                  production        128
2                     métiers        101
3                 compétences         96
4                    stockage         89
5                      enjeux         75
6                      impact         75
7          matières premières         74
8                 entreprises         71
9                 énergétique         59
10                    énergie         55
11                    chaleur         55
12                       plan         51
13                  industrie         45
14      développement durable         43
15                   capacité         42
16                  véhicules         42
17                     master         41
18                  batteries         40
19                     energy         37
20               consommation         36
21                        for         34
22                        all         34
23              

In [6]:
# Filtre manuel pour récupérer que les mots sur lesquels on peut scrapper
mots_a_conserver = [
    'transition écologique',
    'transition ecologique',
    'transition énergétique',
    'transition energetique',
    'énergies nouvelles',
    'nouvelles énergies',
    'énergies renouvelables'
    'matières premières',
    'matieres premieres',
    'enjeu environnemental',
    'enjeux environnementaux',
    'développement durable',
    'efioul',
    'e-fioul',
    'carburant de synthese',
    'carburant de synthèse',
    'biocarburant',
    'bio-carburant',
    'rechauffement climatique',
    'rechauffement climatique',
    'voiture électrique',
    'voitures électriques',
    'véhicule électrique',
    'véhicules électriques',
    'pompe à chaleur',
    'isolation du logement',
    'biochar',
    'éolienne en mer',
    'éoliennes en mer',
    'éolienne offshore',
    'éoliennes offshore',
    'hydrogène',
    'consommation énergétique',
    'sobriété énergétique',
    'entreprises vertes'
]

# Filtrer le dictionnaire de mots en ne conservant que les mots spécifiques
dict_mot_final = dict_mot[dict_mot['Mot'].isin(mots_a_conserver)]
dict_mot_final = dict_mot_final.reset_index(drop=True)
dict_mot_final

,Mot,Fréquence
0,transition écologique,339
1,développement durable,43
2,entreprises vertes,12
3,transition ecologique,999
4,transition énergétique,999
5,transition energetique,999
6,énergies nouvelles,999
7,nouvelles énergies,999
8,matieres premieres,999
9,enjeu environnemental,999


In [7]:
# Construire la liste des mots à rechercher
mots_a_rechercher = dict_mot_final['Mot'].tolist()

# Remplacement des ' 'par '+' pour prératation à la requête 
mots_a_rechercher = [mots.replace(' ', '+') for mots in mots_a_rechercher]
mots_a_rechercher

['transition+écologique',
 'développement+durable',
 'entreprises+vertes',
 'transition+ecologique',
 'transition+énergétique',
 'transition+energetique',
 'énergies+nouvelles',
 'nouvelles+énergies',
 'matieres+premieres',
 'enjeu+environnemental',
 'enjeux+environnementaux',
 'efioul',
 'e-fioul',
 'carburant+de+synthese',
 'carburant+de+synthèse',
 'biocarburant',
 'bio-carburant',
 'rechauffement+climatique',
 'voiture+électrique',
 'voitures+électriques',
 'véhicule+électrique',
 'véhicules+électriques',
 'pompe+à+chaleur',
 'isolation+du+logement',
 'biochar',
 'éolienne+en+mer',
 'éoliennes+en+mer',
 'éolienne+offshore',
 'éoliennes+offshore',
 'hydrogène',
 'consommation+énergétique',
 'sobriété+énergétique']

In [8]:
# ATTENTION, CELA PEUT PRENDRE PLUSIEURS HEURES EN FONCTION DU DE LA TAILLE DU DICTIONNAIRE DE MOTS ET DE LA TEMPORALITE
# Boucle sur chaque mot à rechercher
for mot in mots_a_rechercher:
    # Réinitialiser la date de début pour chaque nouveau mot
    start_date_mot = start_date
    
    # Boucle sur chaque jour
    while start_date_mot <= end_date:
        since_date_mot = start_date_mot.strftime("%Y-%m-%d")
        until_date_mot = (start_date_mot + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        
        # Construire l'URL de la requête avec le mot et les dates du jour en cours
        query = urllib.parse.quote(mot)
        url = f"https://nitter.net/search?f=tweets&q={query}&since={since_date_mot}&until={until_date_mot}&near="
        print(url)

        # Requête HTTP & récupération du contenu de la page (+ ajout d'un header pour contourner la sécurité Nitter)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        req = Request(url, headers=headers)
        response = urlopen(req)
        soup = BeautifulSoup(response.read(), 'html.parser')

        # extraire les informations de chaque tweet pour le jour en cours
        tweet_contents = []
        tweet_dates = []
        tweet_links = []
        tweet_names = []
        tweet_names_id = []
        comment_stats = []
        retweet_stats = []
        quote_stats = []
        heart_stats = []

        for tweet in soup.find_all('div', {'class': 'timeline-item'}):
            # contenu du tweet
            content = tweet.find('div', {'class': 'tweet-content'}).text.strip()
            tweet_contents.append(content)

            # date du tweet
            date = tweet.find('span', {'class': 'tweet-date'}).a['title']
            tweet_dates.append(date)

            # lien du tweet
            link = tweet.find('a', {'class': 'tweet-link'})['href']
            tweet_links.append(link)

            # nom (title) du tweet
            name = tweet.find('a', {'class': 'fullname'})['title']
            tweet_names.append(name)

            # nom (id) du tweet
            name_id = tweet.find('a', {'class': 'username'})['title']
            tweet_names_id.append(name_id)

            # statistiques
            stats = tweet.find_all('div', {'class': 'icon-container'})
            comment_stats.append(stats[0].text.strip())
            retweet_stats.append(stats[1].text.strip())
            quote_stats.append(stats[2].text.strip())
            heart_stats.append(stats[3].text.strip())

        # ajouter les informations du jour en cours à la liste de résultats
        result = pd.DataFrame({
            'Content': tweet_contents,
            'Date': tweet_dates,
            'Link': tweet_links,
            'Utilisateur': tweet_names,
            'Utilisateur_id': tweet_names_id,
            'Comments': comment_stats,
            'Retweets': retweet_stats,
            'Quotes': quote_stats,
            'Hearts': heart_stats
        })
        results.append(result)

        # Passer à la prochaine date
        start_date_mot += datetime.timedelta(days=1)

https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-01&until=2023-01-02&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-02&until=2023-01-03&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-03&until=2023-01-04&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-04&until=2023-01-05&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-05&until=2023-01-06&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-06&until=2023-01-07&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-07&until=2023-01-08&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-08&until=2023-01-09&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-01-09&until=2023-01-10&near=
https://nitter.net/search?f=tweets&q=transitio

https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-03-20&until=2023-03-21&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-03-21&until=2023-03-22&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-03-22&until=2023-03-23&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-03-23&until=2023-03-24&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-03-24&until=2023-03-25&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-03-25&until=2023-03-26&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-03-26&until=2023-03-27&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-03-27&until=2023-03-28&near=
https://nitter.net/search?f=tweets&q=transition%2B%C3%A9cologique&since=2023-03-28&until=2023-03-29&near=
https://nitter.net/search?f=tweets&q=transitio

https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-05&until=2023-01-06&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-06&until=2023-01-07&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-07&until=2023-01-08&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-08&until=2023-01-09&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-09&until=2023-01-10&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-10&until=2023-01-11&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-11&until=2023-01-12&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-12&until=2023-01-13&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-01-13&until=2023-01-14&near=
https://nitter.net/search?f=tweets&q=d%C3%A9ve

https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-03-24&until=2023-03-25&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-03-25&until=2023-03-26&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-03-26&until=2023-03-27&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-03-27&until=2023-03-28&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-03-28&until=2023-03-29&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-03-29&until=2023-03-30&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-03-30&until=2023-03-31&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-03-31&until=2023-04-01&near=
https://nitter.net/search?f=tweets&q=d%C3%A9veloppement%2Bdurable&since=2023-04-01&until=2023-04-02&near=
https://nitter.net/search?f=tweets&q=d%C3%A9ve

https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-09&until=2023-01-10&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-10&until=2023-01-11&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-11&until=2023-01-12&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-12&until=2023-01-13&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-13&until=2023-01-14&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-14&until=2023-01-15&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-15&until=2023-01-16&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-16&until=2023-01-17&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-17&until=2023-01-18&near=
https://nitter.net/search?f=tweets&q=entreprises%2Bvertes&since=2023-01-18&until=2023-01-19&near=
https://nitter.net/s

...

https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-07&until=2023-04-08&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-08&until=2023-04-09&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-09&until=2023-04-10&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-10&until=2023-04-11&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-11&until=2023-04-12&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-12&until=2023-04-13&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-13&until=2023-04-14&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-14&until=2023-04-15&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-15&until=2023-04-16&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-16&until=2023-04-17&near=
https://nitter.net/search?f=tweets&q=hydrog%C3%A8ne&since=2023-04-17&until=2023-

https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-01-28&until=2023-01-29&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-01-29&until=2023-01-30&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-01-30&until=2023-01-31&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-01-31&until=2023-02-01&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-02-01&until=2023-02-02&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-02-02&until=2023-02-03&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-02-03&until=2023-02-04&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-02-04&until=2023-02-05&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-02-

https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-04-10&until=2023-04-11&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-04-11&until=2023-04-12&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-04-12&until=2023-04-13&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-04-13&until=2023-04-14&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-04-14&until=2023-04-15&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-04-15&until=2023-04-16&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-04-16&until=2023-04-17&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-04-17&until=2023-04-18&near=
https://nitter.net/search?f=tweets&q=consommation%2B%C3%A9nerg%C3%A9tique&since=2023-04-

https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-01-19&until=2023-01-20&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-01-20&until=2023-01-21&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-01-21&until=2023-01-22&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-01-22&until=2023-01-23&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-01-23&until=2023-01-24&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-01-24&until=2023-01-25&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-01-25&until=2023-01-26&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-01-26&until=2023-01-27&near=
https://nitter.net/search?f=tweets&q=sob

https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-03-29&until=2023-03-30&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-03-30&until=2023-03-31&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-03-31&until=2023-04-01&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-04-01&until=2023-04-02&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-04-02&until=2023-04-03&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-04-03&until=2023-04-04&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-04-04&until=2023-04-05&near=
https://nitter.net/search?f=tweets&q=sobri%C3%A9t%C3%A9%2B%C3%A9nerg%C3%A9tique&since=2023-04-05&until=2023-04-06&near=
https://nitter.net/search?f=tweets&q=sob

In [17]:
# Concaténer les résultats de chaque jour dans un unique DataFrame
df_all_tweets = pd.concat(results)

In [18]:
df_all_tweets

,Content,Date,Link,Utilisateur,Utilisateur_id,Comments,Retweets,Quotes,Hearts
0,"Ah, en effet...\nLa transition écologique est ...","Jan 1, 2023 · 11:52 PM UTC",/PaysagesIroise/status/1609699207169941505#m,Sauvegarde Paysages d'Iroise,@PaysagesIroise,,,,1
1,Méthode pour une transition écologique dans le...,"Jan 1, 2023 · 11:29 PM UTC",/InfosEducation/status/1609693388776906752#m,Education & Formation,@InfosEducation,,,,
2,Enseignement agricole et transition écologique...,"Jan 1, 2023 · 11:03 PM UTC",/VIENSflashmober/status/1609686785084493826#m,Etat d'urgence,@VIENSflashmober,,,,
3,la pollueuse de notre planète qui a des intérê...,"Jan 1, 2023 · 10:26 PM UTC",/LIBERTE1311/status/1609677427223007232#m,eddy,@LIBERTE1311,,,,1
4,Quand la transition écologique bouscule la pol...,"Jan 1, 2023 · 10:21 PM UTC",/PACOLLOT/status/1609676117555781634#m,Pierre-Alain Collot,@PACOLLOT,,,,
...,...,...,...,...,...,...,...,...,...
15,#decarbononsSeineEure\n\n3eme table ronde sur ...,"Jun 1, 2023 · 9:49 AM UTC",/LePoleTES/status/1664207513951109121#m,Pôle TES,@LePoleTES,,2,,4
16,Le numérique 🖥️ peut-il soutenir la #sobriété...,"Jun 1, 2023 · 9:10 AM UTC",/FLievyn/status/1664197785166438402#m,Florence.Lievyn,@FLievyn,,1,,3
17,Ils prouvent que le monde sera à l'heure sur l...,"Jun 1, 2023 · 8:55 AM UTC",/GlennAndr7/status/1664193820966150144#m,Glenn André,@GlennAndr7,1,,,2
18,"Acteur de la ressource en eau, promoteur d’un ...","Jun 1, 2023 · 8:24 AM UTC",/vnf_officiel/status/1664186202155634688#m,VNF - Voies navigables de France,@vnf_officiel,,3,,1


In [19]:
# Fonction pour nettoyer les dates
def clean_date(date_string):
    if pd.isnull(date_string):
        return date_string
    cleaned_string = date_string.replace("¬∑", "")
    cleaned_string = date_string.replace("Â·", "")
    cleaned_string = date_string.replace("Ā", "")
    cleaned_string = date_string.replace("·", "")
    return cleaned_string.strip()

# Nettoyer la colonne 'Date'
df_all_tweets['Date'] = df_all_tweets['Date'].apply(lambda x: clean_date(x))

# Convertir la colonne 'Date' en format datetime
df_all_tweets['Date'] = pd.to_datetime(df_all_tweets['Date'], format="%b %d, %Y %I:%M %p UTC", errors='coerce')

# Créer les colonnes 'Jour', 'Mois' et 'Année'
df_all_tweets['Jour'] = df_all_tweets['Date'].dt.day
df_all_tweets['Mois'] = df_all_tweets['Date'].dt.month
df_all_tweets['Année'] = df_all_tweets['Date'].dt.year

# Convertir les colonnes 'Jour', 'Mois' et 'Année' en entiers
df_all_tweets['Jour'] = df_all_tweets['Jour'].apply(lambda x: str(int(x)) if not pd.isnull(x) else '')
df_all_tweets['Mois'] = df_all_tweets['Mois'].apply(lambda x: str(int(x)) if not pd.isnull(x) else '')
df_all_tweets['Année'] = df_all_tweets['Année'].apply(lambda x: str(int(x)) if not pd.isnull(x) else '')

# Convertir la colonne 'Date' en format JJ/MM/AAAA
df_all_tweets['Date'] = df_all_tweets['Date'].dt.strftime("%d/%m/%Y")


In [20]:
df_all_tweets = df_all_tweets.reindex(columns=['Content', 'Date', 'Jour', 'Mois', 'Année', 'Link', 'Utilisateur', 'Utilisateur_id', 'Comments', 'Retweets', 'Quotes', 'Hearts'])

In [59]:
df_all_tweets

,Content,Date,Jour,Mois,Année,Link,Utilisateur,Utilisateur_id,Comments,Retweets,Quotes,Hearts
0,"Ah, en effet...\nLa transition écologique est ...",01/01/2023,1,1,2023,/PaysagesIroise/status/1609699207169941505#m,Sauvegarde Paysages d'Iroise,@PaysagesIroise,,,,1
1,Méthode pour une transition écologique dans le...,01/01/2023,1,1,2023,/InfosEducation/status/1609693388776906752#m,Education & Formation,@InfosEducation,,,,
2,Enseignement agricole et transition écologique...,01/01/2023,1,1,2023,/VIENSflashmober/status/1609686785084493826#m,Etat d'urgence,@VIENSflashmober,,,,
3,la pollueuse de notre planète qui a des intérê...,01/01/2023,1,1,2023,/LIBERTE1311/status/1609677427223007232#m,eddy,@LIBERTE1311,,,,1
4,Quand la transition écologique bouscule la pol...,01/01/2023,1,1,2023,/PACOLLOT/status/1609676117555781634#m,Pierre-Alain Collot,@PACOLLOT,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
15,#decarbononsSeineEure\n\n3eme table ronde sur ...,01/06/2023,1,6,2023,/LePoleTES/status/1664207513951109121#m,Pôle TES,@LePoleTES,,2,,4
16,Le numérique 🖥️ peut-il soutenir la #sobriété...,01/06/2023,1,6,2023,/FLievyn/status/1664197785166438402#m,Florence.Lievyn,@FLievyn,,1,,3
17,Ils prouvent que le monde sera à l'heure sur l...,01/06/2023,1,6,2023,/GlennAndr7/status/1664193820966150144#m,Glenn André,@GlennAndr7,1,,,2
18,"Acteur de la ressource en eau, promoteur d’un ...",01/06/2023,1,6,2023,/vnf_officiel/status/1664186202155634688#m,VNF - Voies navigables de France,@vnf_officiel,,3,,1


In [58]:
import csv

df_all_tweets.to_csv(
    'PPD_ALL_Scrapping.csv',
    sep=';',
    index=False,
    encoding='utf-8-sig',
    #quoting=csv.QUOTE_NONE,
    #escapechar='\\'
)

In [ ]:
df_all_tweets.to_excel(r'PPD_ALL_Scrapping.xlsx', index=False)

# Selenium - Code avec 1 jour (demo)

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [42]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://nitter.net/search?f=tweets&q=transition+%C3%A9nerg%C3%A9tique+energetique&since=2023-04-20&until=2023-04-21&near=")
html = driver.page_source

In [43]:
soup = BeautifulSoup(html, 'html.parser')

In [44]:
# extraire les informations de chaque tweet
tweet_contents = []
tweet_dates = []
tweet_links = []
tweet_names = []
comment_stats = []
retweet_stats = []
quote_stats = []
heart_stats = []

for tweet in soup.find_all('div', {'class': 'timeline-item'}):
    # contenu du tweet
    content = tweet.find('div', {'class': 'tweet-content'}).text.strip()
    tweet_contents.append(content)
    
    # date du tweet
    date = tweet.find('span', {'class': 'tweet-date'}).a['title']
    tweet_dates.append(date)
    
    # lien du tweet
    link = tweet.find('a', {'class': 'tweet-link'})['href']
    tweet_links.append(link)
    
    # nom (title) du tweet
    name = tweet.find('a', {'class': 'fullname'})['title']
    tweet_names.append(name)
    
    # statistiques
    stats = tweet.find_all('div', {'class': 'icon-container'})
    comment_stats.append(stats[0].text.strip())
    retweet_stats.append(stats[1].text.strip())
    quote_stats.append(stats[2].text.strip())
    heart_stats.append(stats[3].text.strip())

In [45]:
# créer un dataframe pandas à partir des listes d'informations
df = pd.DataFrame({
    'Content': tweet_contents,
    'Date': tweet_dates,
    'Link': tweet_links,
    'Utilisateur': tweet_names,
    'Comments': comment_stats,
    'Retweets': retweet_stats,
    'Quotes': quote_stats,
    'Hearts': heart_stats
})

In [46]:
df

,Content,Date,Link,Utilisateur,Comments,Retweets,Quotes,Hearts
0,"RATP (Bus)\n\n🌿Depuis le 19/04/23, la ligne 14...","Apr 20, 2023 · 11:14 PM UTC",/chanstransports/status/1649189726593732608#m,Claude-Henry & ses périples,2,6,1,47
1,"Le rapport rendu ce vendredi à @AgnesRunacher,...","Apr 20, 2023 · 10:45 PM UTC",/ChamboncelBen/status/1649182448842833921#m,Benoît Chamboncel,,,22,1
2,📢 Défis de l'employabilité avec la transition ...,"Apr 20, 2023 · 10:19 PM UTC",/IFDD_OIF/status/1649176003158192128#m,IFDD,,2,,5
3,« C’est un pied de nez au Québec. On se fait d...,"Apr 20, 2023 · 9:22 PM UTC",/seblemire/status/1649161573867352066#m,Sébastien Lemire,2,,,1
4,Chère @_EvaSas En #Allemagne ils n'ont pas att...,"Apr 20, 2023 · 8:56 PM UTC",/PIERREMAURIN/status/1649154996355624967#m,Pierre Maurin,,1,,2
5,📃 Le 1er conseil d’administration de la Fondat...,"Apr 20, 2023 · 7:11 PM UTC",/FondationOpenC/status/1649128606319214593#m,Fondation OPEN-C,1,1,,4
6,Métaux stratégiques de l'Afrique pour la trans...,"Apr 20, 2023 · 6:56 PM UTC",/energiemines/status/1649124953923387417#m,Magazine Energie,,,,
7,il est juste que les acteurs publics[…]soient ...,"Apr 20, 2023 · 6:49 PM UTC",/JCondamine/status/1649123244178276383#m,Jean Condamine 🇫🇷 🇪🇸 🇬🇧 👍🍻🍺🤔🧐😉👌🥂🍾,2,5,,22
8,Roland Lescure et de la ministre de la transit...,"Apr 20, 2023 · 6:20 PM UTC",/DanielM11626471/status/1649115758109007875#m,Daniel Moore 🇷🇺🇷🇺🇷🇺,1,1,,4
9,"Plus de 10 millions de morts, au moins 500 000...","Apr 20, 2023 · 6:20 PM UTC",/EpopoYanick/status/1649115722583359492#m,Epopo yanick,,1,,1


In [47]:
# Fonction pour convertir la date dans le bon format
def convert_date(date_string):
    date_obj = datetime.datetime.strptime(date_string.split(' · ')[0], "%b %d, %Y")
    return date_obj.strftime("%d/%m/%Y")

# Convertir la colonne 'Date' en format datetime
df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: x.split(' · ')[0]), format='%b %d, %Y')

# Créer les colonnes 'Jour', 'Mois' et 'Année'
df['Jour'] = df['Date'].dt.day
df['Mois'] = df['Date'].dt.month
df['Année'] = df['Date'].dt.year

# Convertir la colonne 'Date' en format JJ/MM/AAAA
df['Date'] = df['Date'].apply(lambda x: convert_date(x.strftime("%b %d, %Y")))

In [48]:
df = df.reindex(columns=['Content', 'Date', 'Jour', 'Mois', 'Année', 'Link', 'Utilisateur','Comments', 'Retweets', 'Quotes', 'Hearts'])

In [49]:
df

,Content,Date,Jour,Mois,Année,Link,Utilisateur,Comments,Retweets,Quotes,Hearts
0,"RATP (Bus)\n\n🌿Depuis le 19/04/23, la ligne 14...",20/04/2023,20,4,2023,/chanstransports/status/1649189726593732608#m,Claude-Henry & ses périples,2,6,1,47
1,"Le rapport rendu ce vendredi à @AgnesRunacher,...",20/04/2023,20,4,2023,/ChamboncelBen/status/1649182448842833921#m,Benoît Chamboncel,,,22,1
2,📢 Défis de l'employabilité avec la transition ...,20/04/2023,20,4,2023,/IFDD_OIF/status/1649176003158192128#m,IFDD,,2,,5
3,« C’est un pied de nez au Québec. On se fait d...,20/04/2023,20,4,2023,/seblemire/status/1649161573867352066#m,Sébastien Lemire,2,,,1
4,Chère @_EvaSas En #Allemagne ils n'ont pas att...,20/04/2023,20,4,2023,/PIERREMAURIN/status/1649154996355624967#m,Pierre Maurin,,1,,2
5,📃 Le 1er conseil d’administration de la Fondat...,20/04/2023,20,4,2023,/FondationOpenC/status/1649128606319214593#m,Fondation OPEN-C,1,1,,4
6,Métaux stratégiques de l'Afrique pour la trans...,20/04/2023,20,4,2023,/energiemines/status/1649124953923387417#m,Magazine Energie,,,,
7,il est juste que les acteurs publics[…]soient ...,20/04/2023,20,4,2023,/JCondamine/status/1649123244178276383#m,Jean Condamine 🇫🇷 🇪🇸 🇬🇧 👍🍻🍺🤔🧐😉👌🥂🍾,2,5,,22
8,Roland Lescure et de la ministre de la transit...,20/04/2023,20,4,2023,/DanielM11626471/status/1649115758109007875#m,Daniel Moore 🇷🇺🇷🇺🇷🇺,1,1,,4
9,"Plus de 10 millions de morts, au moins 500 000...",20/04/2023,20,4,2023,/EpopoYanick/status/1649115722583359492#m,Epopo yanick,,1,,1


# Selenium - Code pour plusieurs jours

In [5]:
pip install webdriver_manager

In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [52]:
# Date de début de la recherche
start_date = datetime.date(2023, 5, 1)

# Date de fin de la recherche
end_date = datetime.date(2023, 5, 2)

# Créer la liste des dates à parcourir
delta = datetime.timedelta(days=1)
dates = []
while start_date <= end_date:
    dates.append(start_date)
    start_date += delta

In [54]:
# Initialiser le webdriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Liste des données à récupérer
tweet_contents = []
tweet_dates = []
tweet_links = []
tweet_names = []
comment_stats = []
retweet_stats = []
quote_stats = []
heart_stats = []

In [55]:
# Boucle pour parcourir les pages pour chaque date
for date in dates:
    url = f"https://nitter.net/search?f=tweets&q=transition+%C3%A9nerg%C3%A9tique+energetique&since={date}&until={date+delta}&near="
    driver.get(url)
    while True:
        # Extraire les informations de chaque tweet sur la page
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        for tweet in soup.find_all('div', {'class': 'timeline-item'}):
            # Contenu du tweet
            content = tweet.find('div', {'class': 'tweet-content'}).text.strip()
            tweet_contents.append(content)
            
            # Date du tweet
            date = tweet.find('span', {'class': 'tweet-date'}).a['title']
            tweet_dates.append(date)
            
            # Lien du tweet
            link = tweet.find('a', {'class': 'tweet-link'})['href']
            tweet_links.append(link)
            
            # Nom (title) du tweet
            name = tweet.find('a', {'class': 'fullname'})['title']
            tweet_names.append(name)
            
            # Statistiques
            stats = tweet.find_all('div', {'class': 'icon-container'})
            comment_stats.append(stats[0].text.strip())
            retweet_stats.append(stats[1].text.strip())
            quote_stats.append(stats[2].text.strip())
            heart_stats.append(stats[3].text.strip())
        
        # Cliquer sur le bouton "Load more" si présent
        try:
            #load_more_button = driver.find_element_by_class_name("show-more").find_element_by_tag_name("a")
            #load_more_button = driver.find_elements(By.CLASS_NAME, "show-more")
            #load_more_button=load_more_button.find_elements(By.TAG_NAME, "a")
            #print(load_more_button)
            #load_more_button.click()
            #print(load_more_button[0].text)
            #driver.execute_script("arguments[0].click();", load_more_button);
            
            # get element 
            element = driver.find_element_by_link_text("Load more")
            print(element)
            # click the element
            element.click()
            
        except:
            break

In [56]:
# Créer un dataframe pandas avec les données récupérées
df = pd.DataFrame({
    'Content': tweet_contents,
    'Date': tweet_dates,
    'Link': tweet_links,
    'Utilisateur': tweet_names,
    'Comments': comment_stats,
    'Retweets': retweet_stats,
    'Quotes': quote_stats,
    'Hearts': heart_stats
})

In [57]:
# Fonction pour convertir la date dans le bon format
def convert_date(date_string):
    date_obj = datetime.datetime.strptime(date_string.split(' · ')[0], "%b %d, %Y")
    return date_obj.strftime("%d/%m/%Y")

# Convertir la colonne 'Date' en format datetime
df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: x.split(' · ')[0]), format='%b %d, %Y')

# Créer les colonnes 'Jour', 'Mois' et 'Année'
df['Jour'] = df['Date'].dt.day
df['Mois'] = df['Date'].dt.month
df['Année'] = df['Date'].dt.year

# Convertir la colonne 'Date' en format JJ/MM/AAAA
df['Date'] = df['Date'].apply(lambda x: convert_date(x.strftime("%b %d, %Y")))

In [58]:
df = df.reindex(columns=['Content', 'Date', 'Jour', 'Mois', 'Année', 'Link', 'Utilisateur','Comments', 'Retweets', 'Quotes', 'Hearts'])

In [59]:
df

,Content,Date,Jour,Mois,Année,Link,Utilisateur,Comments,Retweets,Quotes,Hearts
0,Pourquoi le Maroc est important pour la transi...,01/05/2023,1,5,2023,/Rozlbrahim/status/1653155503642402821#m,인생은 계속됩니다,1,,,1
1,dans le sable du désert - et des plans ambitie...,01/05/2023,1,5,2023,/Rozlbrahim/status/1653155505852801024#m,인생은 계속됩니다,1,,,1
2,Repenser la transition énergétique : Le menson...,01/05/2023,1,5,2023,/PEGE_french/status/1653127278828691457#m,Mösl Roland,,,,
3,"Là j'approuve, ça n'est pas une oeuvre d'art q...",01/05/2023,1,5,2023,/marco_il_biondo/status/1653119911894327302#m,Marco il Biondo,,,,
4,"Oui, mais non puisque l'Allemagne se donne les...",01/05/2023,1,5,2023,/Djoanarcho/status/1653111091021463552#m,BZH 🏴‍☠️🏴,,,,
5,Pas la peine de demander qui vous suivez polit...,01/05/2023,1,5,2023,/ceditpro/status/1653101214333034514#m,cedric,,,,
6,Participez à la transition énergétique avec ce...,01/05/2023,1,5,2023,/GreenBois_fr/status/1653093514144538632#m,Green Bois,,,,
7,"mes félicitations ! votre livre, très utile po...",01/05/2023,1,5,2023,/michel_Halpes/status/1653090639108620289#m,Michel Cranga climat et responsabilité,2,,,3
8,"Besoin en investissement, en travailleurs, con...",01/05/2023,1,5,2023,/FR_Conversation/status/1653090552382992384#m,The Conversation France,,,,2
9,"""transition écologique"", ou encore ""transition...",01/05/2023,1,5,2023,/5gLinky/status/1653084728046632978#m,Stop Linky-5G Montpellier,,,,2


# Brute force acteurs - Stats des acteurs (MAJ suite à sécurité Nitter)

In [29]:
# Chargement du dictionnaire des acteurs
dict_acteurs = df_all_tweets

# Garder que les utilisateurs uniques
dict_acteurs_unique = dict_acteurs.drop_duplicates(subset=['Utilisateur_id'])['Utilisateur_id'].to_list()
dict_acteurs_unique

['@PaysagesIroise',
 '@InfosEducation',
 '@VIENSflashmober',
 '@LIBERTE1311',
 '@PACOLLOT',
 '@verolive83',
 '@Hergastul',
 '@PatrickGlesener',
 '@giOrdanoB6946',
 '@randomie7i',
 '@Marie08979035',
 '@JeannotMbaya',
 '@latulipenoire31',
 '@Olivancalderone',
 '@MySweetimmo',
 '@LaurencePalais',
 '@PMonoxyle',
 '@MendiniNassim',
 '@DavidBorie4',
 '@BARLANicolas1',
 '@Ortofon76',
 '@Prefet49',
 '@sophie_bussiere',
 '@lemainelibre',
 '@Actuenviro',
 '@AudreyHenocque',
 '@eco_actu',
 '@SartheInfos',
 '@groum_groum',
 '@Babapolitique',
 '@Q_deBremond',
 '@RomainLaveau',
 '@FrexitOtanExit',
 '@Cesarmand',
 '@o_environnement',
 '@LaTribune',
 '@LCI',
 '@imperma1914',
 '@ChristopheBechu',
 '@FredBesnard',
 '@Trunkhanwow',
 '@UnivDroit_EDD',
 '@TLConseil',
 '@favier_florent',
 '@deslnfor',
 '@Benjam1Lucas',
 '@shilderi',
 '@Vivien_PI',
 '@DorianBianco07',
 '@LouisLghni',
 '@Lejeune_paris',
 '@communiquaction',
 '@FrancoisBulloc1',
 '@jeremyerline',
 '@EadgGdea',
 '@FabriceRivet',
 '@SachaMaunier

In [32]:
# Vérifie si le premier caractère est un @ et le supprime, sinon garde l'ID
dict_acteurs_unique = [
    char[1:] if char.startswith('@')
    else char for char in dict_acteurs_unique
]

In [46]:
dict_acteurs_unique

['PaysagesIroise',
 'InfosEducation',
 'VIENSflashmober',
 'LIBERTE1311',
 'PACOLLOT',
 'verolive83',
 'Hergastul',
 'PatrickGlesener',
 'giOrdanoB6946',
 'randomie7i',
 'Marie08979035',
 'JeannotMbaya',
 'latulipenoire31',
 'Olivancalderone',
 'MySweetimmo',
 'LaurencePalais',
 'PMonoxyle',
 'MendiniNassim',
 'DavidBorie4',
 'BARLANicolas1',
 'Ortofon76',
 'Prefet49',
 'sophie_bussiere',
 'lemainelibre',
 'Actuenviro',
 'AudreyHenocque',
 'eco_actu',
 'SartheInfos',
 'groum_groum',
 'Babapolitique',
 'Q_deBremond',
 'RomainLaveau',
 'FrexitOtanExit',
 'Cesarmand',
 'o_environnement',
 'LaTribune',
 'LCI',
 'imperma1914',
 'ChristopheBechu',
 'FredBesnard',
 'Trunkhanwow',
 'UnivDroit_EDD',
 'TLConseil',
 'favier_florent',
 'deslnfor',
 'Benjam1Lucas',
 'shilderi',
 'Vivien_PI',
 'DorianBianco07',
 'LouisLghni',
 'Lejeune_paris',
 'communiquaction',
 'FrancoisBulloc1',
 'jeremyerline',
 'EadgGdea',
 'FabriceRivet',
 'SachaMaunier',
 'JuliaCerisi',
 'melggo',
 'iwona46',
 'ChaBahuon',
 

In [63]:
dict_acteurs_unique2 = dict_acteurs_unique[:10000]

In [64]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [66]:
# ATTENTION, CELA PEUT PRENDRE PLUSIEURS HEURES EN FONCTION DU NOMBRE D'ACTEURS A SCRAPPER
# Liste pour stocker les résultats
profile_results = []

# Boucle sur chaque utilisateur
for user_id in dict_acteurs_unique2:
    # Construire l'URL du profil de l'utilisateur
    url = f"https://nitter.net/{user_id}"
    
    # Requête HTTP & récupération du contenu de la page (+ ajout d'un header pour contourner la sécurité Nitter)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    req = Request(url, headers=headers)
    response = urlopen(req)
    soup = BeautifulSoup(response.read(), 'html.parser')
    
    # Extraire les informations du profil
    utilisateur_name = soup.find('a', {'class': 'profile-card-fullname'}).text.strip()
    profile_bio_element = soup.find('div', {'class': 'profile-bio'})
    profile_bio = profile_bio_element.text.strip() if profile_bio_element is not None else ""
    tweets_count = soup.find('li', {'class': 'posts'}).find('span', {'class': 'profile-stat-num'}).text.strip()
    following_count = soup.find('li', {'class': 'following'}).find('span', {'class': 'profile-stat-num'}).text.strip()
    followers_count = soup.find('li', {'class': 'followers'}).find('span', {'class': 'profile-stat-num'}).text.strip()
    likes_count = soup.find('li', {'class': 'likes'}).find('span', {'class': 'profile-stat-num'}).text.strip()
    
    # Supprimer les virgules dans les métriques
    tweets_count = tweets_count.replace(",", "")
    following_count = following_count.replace(",", "")
    followers_count = followers_count.replace(",", "")
    likes_count = likes_count.replace(",", "")
    
    # Ajouter les informations du profil à la liste des résultats
    profile_results.append({
        'Utilisateur_id': user_id,
        'Utilisateur_name': utilisateur_name,
        'Profile_bio': profile_bio,
        'Tweets': tweets_count,
        'Following': following_count,
        'Followers': followers_count,
        'Likes': likes_count
    })
    print(f"Index: {index}, User ID: {user_id}")

# Créer un DataFrame à partir des résultats
dict_acteurs_final = pd.DataFrame(profile_results)

PaysagesIroise
InfosEducation
VIENSflashmober
LIBERTE1311
PACOLLOT


Exception ignored in: <function ZipFile.__del__ at 0x000001F6D0B33A60>
Traceback (most recent call last):
  File "C:\Users\FTG1\AppData\Local\Programs\Python\Python311\Lib\zipfile.py", line 1870, in __del__
    self.close()
  File "C:\Users\FTG1\AppData\Local\Programs\Python\Python311\Lib\zipfile.py", line 1887, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


verolive83
Hergastul
PatrickGlesener
giOrdanoB6946
randomie7i
Marie08979035
JeannotMbaya
latulipenoire31
Olivancalderone
MySweetimmo
LaurencePalais
PMonoxyle
MendiniNassim
DavidBorie4
BARLANicolas1
Ortofon76
Prefet49
sophie_bussiere
lemainelibre
Actuenviro
AudreyHenocque
eco_actu
SartheInfos
groum_groum
Babapolitique
Q_deBremond
RomainLaveau
FrexitOtanExit
Cesarmand
o_environnement
LaTribune
LCI
imperma1914
ChristopheBechu
FredBesnard
Trunkhanwow
UnivDroit_EDD
TLConseil
favier_florent
deslnfor
Benjam1Lucas
shilderi
Vivien_PI
DorianBianco07
LouisLghni
Lejeune_paris
communiquaction
FrancoisBulloc1
jeremyerline
EadgGdea
FabriceRivet
SachaMaunier
JuliaCerisi
melggo
iwona46
ChaBahuon
antoinecaplan
HachaniNaima
m_Romagnani
LibertyLoveFr
reix_toine
JeunesseNupes
Antoine0667Best
Arr_Verdun
DarkPhoenix9666
LP641
jnbarrot
RDelinfo
NoamLeandri
benoitbonte
VeloBucVGP
VonbraunWerner
Firenault
claran55
Goulran_
baceau
ademe
ChristVdbPro
Madininair972
systemcruyff
escaledenuit
Jean_LucFUGIT
24matins


PFRunner
Amoklaufer
TelegrammeGGP
fidaltys
AzzazNadege
BonnevayRemy
YoshYosh59
ollybret
ASSURANCESMEYER
ID_LinfoDurable
mclucal
Castielsewhere
Heyjuanito1
Loraineroncin
JulesWP
clement_sas
GanArbois
al_secco
Cabinet_Cogec
CX_laurent
gan_arriat
sebgdu
AromaSencha
DayanetExperts
USBEKetRICA
b_etche
Assu_Charlet
hostilus
Le_Figaro
Gabrielc6472827
DeParfouru
DawnyCedric
1669770000a
AureganYves
SaiyanBio
Denis_Bapt
visant_dessins
lionel_thual
CouleursParis
publicsenat
MedefPDL
EricBezouSUD
alain_co
AlexB55170823
RmdCoco
Mediasinfos
TomekTR
davidfayon
sd2776riseup
GuillaumeQ1T1
heubeu31
MaldororX
RMCInfo
MALHERBEL
MarieMagda6
ise_uqam
BenjaminBlancBB
rougesgorges
JN_Gibbs
AiretCosmos
GChaussemy
etdoncalors
EmmanuelCapus
ming_lilian
jeremydumont
ClaudeLENGLET
NainMangeable
sidibemacisse
DuteilMyriam
gruffat_claude
franziska_mnhrz
becht_olivier
Farida_Amrani_
Megoustastou
Ecophanie
lovizinziair
montreal110
Emile_Doigts
memoristic
LAutreCentre
MartinezBoloss
AsnieresPcf
droitdelacom
CelsaAlumni

AxeC975
ManuelDUPONT7
J_M_Jacques
herisson13
PhilJLau
ClairePinto91
nowoxo
BretonLorrain
EmmaVelo12
InfoVoisin
frdossant
climatquestions
Renaissance7_95
LouChesne
ClaudineClodell
therealyeahmen
laydgeur
jmt_musique
BelaidGabriela
michelrocard70
ChArrighi
zirealvipi
MonkeyDFrite
NejmaBrahim
kharumtoupitoup
TLGLePtitGars
ThomasChatelard
iamrayanhcn_
nicolasbissaud
MickaCorreia
UniverSallaG
EdenVal1
babonnay
hotdogsmayo
jfrial
rkivehirai
IronPhil7
Lesecologistes1
lopinion_eco
GuergBen
MTL_VSP
PaulMul97561773
Adada_Hue
BFMMarseille
aphm_actu
mleeut
natsilbert
pascallepetit1
DraduagF
DAmoursMath
ymbaechler
Alexchretien74
rolandrichard
alexmenuel
gdcurieux
MicheleRubirola
ClemSenechal
KaraHibe
julienbayou
fgmontreal
OlivierSchneid
joroy
sycfi
Adamniel11
Legifrance1
XB_Tiplouf
VERMEULENEvely2
PREDIGAN
ITrippenbach
Mercilavie1
lisse_ane
bcassoret
AlamedaLamelune
marylinebaumard
DanieleDoucet
Vivet_Lili
AgostiniCome
Richard_Marion_
ngdoc1
EricMuraille
LeFnake
JM78394748372
JeanneNeymar
fabienvi

...

Hoffmann_Green
Carol_Ann101
masterHCPupjv
DroitScPoNice
WNaitMazi
payplug
CampusMer
florimond_p
lavenir_net
BX1_Actu
IrocoDigital
GroupeLuminess
VINCIAutoroutes
bpetitjean
lachaineaudio
Keuv_
PlasticOdyssey
SocBotaniqueF
MEIJIcom
HazizaEmma
sufcaUBO
barmeimmo
PouvoirDeVivre
Marie31918177
LaMadeleine_49
LeVif
ENSEIRBMATMECA
_Oxwin_
Estaca_twit
FondationSNCF
kereonnec
Armees_Gouv
BordeauxINP
Mondjolo
groupeQUALITEL
ClubAudace
AXAFrance
PYJeholet
InstitutFMES
EVE_Transport
Calys84701718
PanettonePazzo
pageimpaire
InesNjonpang
DESANGOSSE
RaphieJarrige
uniscite
leprincematthi2
Archives_94
ClubsterNSL
navalgroup
FiboisBretagne
alternatiba69
Aema_Groupe
TerriAVivreS
mickaelberthout
WebsCorneille
CentraleLyon
SINGSAGabon
IngridNappi
GacemLatifa
alex_rousseau7
simpforiderr
tohruIover
LttaC
sylchery
CCI_Paris_IdF
MauzeGreg
BiscuitsGateaux
BD_Group
HESAM_Univ
AFHalifax
tuba_lyon
lemilaparis
squirrel75_
fne_mp
prodiss
ArchimagRedac
mh_agard
IPSA
SAS_SabiAgri
Larcheresavocat
Veronique_immo
RolandRa

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Papineau_LJ
mproulx21


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


PaulPlamondon
ElodieVercken
OGE_geometres
ECCCSciTech
ser_enr
AudericMaret
TurgeonRodrigue
LefrancoisLou2
sallesleMonde
Forumeteoclimat
abadinte
FeronSabrina
JF_Trucchi
AusoniusEdit
AG2RLAMONDIALE
CENOZO_Afrique
JeanLemire
ctcpa_cti
Anses_fr
Midi2i
BrunoCoupas
OlivierFJB
Jagisamonechel1
StopTERRA2
fdsea53
PatrimoniaC
lemaym91
essec
too_sham
AlphaWhisky67
PresseDILA
daniele_favari
departement13
hautsdefrance
BDomergue
RupturEcoBleue
MINES_StEtienne
BLB_Vignobles
jmjoannes
metropolerennes
clugery
MarioUnivRsalis
ACXIAS
NiortNumeric
sylvainrousse10
architectes_idf
SOLIDEO_JOP
ConstasMagazine
PierreMoninIDF
CThomassinPP
RtsMonaco
metrop_davenir
Cdefi
CathCervoni
DILA_officiel
NebedaySenegal
Enroweb
De_lege_feranda
Droitinccom
InstitutAgroRA
MarieMoerman
diegohidalgodem
celui_qui_est
UriopssOC
sin4_m
LeBoursicoteure
lesoutremer
ImperiumMarche
EmergencyOng
quebeccongres
DthinkerNetwork
vanessamilewski
bernardsolidair
canopea_be
LeChat58145491
MediasFrance
JJBatardiere
HWillert
acayed61
Prozo

In [67]:
# Convertir les colonnes en type numérique
dict_acteurs_final['Tweets'] = pd.to_numeric(dict_acteurs_final['Tweets'], errors='coerce')
dict_acteurs_final['Following'] = pd.to_numeric(dict_acteurs_final['Following'], errors='coerce')
dict_acteurs_final['Followers'] = pd.to_numeric(dict_acteurs_final['Followers'], errors='coerce')
dict_acteurs_final['Likes'] = pd.to_numeric(dict_acteurs_final['Likes'], errors='coerce')

# Ajout d'une colonne score qui est la somme des métriques Likes, Followers, Following et Tweets
dict_acteurs_final['Score'] = dict_acteurs_final['Likes'] + dict_acteurs_final['Followers'] + dict_acteurs_final['Following'] + dict_acteurs_final['Tweets']
dict_acteurs_final

,Utilisateur_id,Utilisateur_name,Profile_bio,Tweets,Following,Followers,Likes,Score
0,PaysagesIroise,Sauvegarde Paysages d'Iroise,Association Sauvegarde Paysages d'Iroise (Fini...,23213,159,480,22396,46248
1,InfosEducation,Education & Formation,Veille éducation - #Formation Professionnel - ...,26018,9145,8522,16,43701
2,VIENSflashmober,Etat d'urgence,"Insurrection permanente, alternative à Punishm...",1011,126,19,111,1267
3,LIBERTE1311,eddy,,90381,114,244,19719,110458
4,PACOLLOT,Pierre-Alain Collot,Enseignant-chercheur - Institut Champollion - ...,1196,293,243,897,2629
...,...,...,...,...,...,...,...,...
9995,aliptic_,ALIPTIC,Association #Limousine des Professionnels des ...,3684,645,1969,2578,8876
9996,iae_bordeaux,IAE Bordeaux,"Twitter officiel de l'IAE Bordeaux, l'Ã©cole d...",885,173,793,411,2262
9997,sprinosrk,sprino,fly me to the moon,65495,361,298,32526,98680
9998,aticbernard,bernard crozel,amoureux de la terre des arbres des oiseaux et...,10745,235,205,7341,18526


In [86]:
dict_acteurs_final_desc = dict_acteurs_final.sort_values('Score', ascending=False)
dict_acteurs_final_desc

,Utilisateur_id,Utilisateur_name,Profile_bio,Tweets,Following,Followers,Likes,Score
324,lemondefr,Le Monde,L'actualité de référence par la rédaction du M...,445119,671,10646193,1701,11093684
3827,JustinTrudeau,Justin Trudeau,"Father, husband, @liberal_party Leader, 23rd P...",44258,920,6459787,131,6505096
7933,RFI,RFI,Suivez toute l'actualité du monde et de l'Afri...,402665,782,4340936,4312,4748695
1791,BFMTV,BFMTV,Première sur l'info Retrouvez-nous sur Snapcha...,721702,560,3982204,1718,4706184
4166,afpfr,Agence France-Presse,"Morceaux choisis des fils d'actu, coulisses & ...",464057,1366,4089546,103,4555072
...,...,...,...,...,...,...,...,...
4291,EspereCom,Espere Sénégal,,2,4,0,0,6
276,ThomasL18174027,Citylights,,1,3,0,0,4
5715,NormanWhea45693,₿ A R I N E K A ✪🅧,Randall M. Barabas III,3,0,0,0,3
9785,AssociationADE5,Association ADECC,,1,0,0,0,1


In [83]:
#dict_acteurs_final_desc_100 = dict_acteurs_final_desc.head(100)
#dict_acteurs_final_desc_100

,Utilisateur_id,Utilisateur_name,Profile_bio,Tweets,Following,Followers,Likes,Score
324,lemondefr,Le Monde,L'actualité de référence par la rédaction du M...,445119,671,10646193,1701,11093684
3827,JustinTrudeau,Justin Trudeau,"Father, husband, @liberal_party Leader, 23rd P...",44258,920,6459787,131,6505096
7933,RFI,RFI,Suivez toute l'actualité du monde et de l'Afri...,402665,782,4340936,4312,4748695
1791,BFMTV,BFMTV,Première sur l'info Retrouvez-nous sur Snapcha...,721702,560,3982204,1718,4706184
4166,afpfr,Agence France-Presse,"Morceaux choisis des fils d'actu, coulisses & ...",464057,1366,4089546,103,4555072
...,...,...,...,...,...,...,...,...
721,publicsenat,Public Sénat,La chaîne politique et parlementaire. TNT cana...,153208,484,419270,3599,576561
9753,DDDamiennn,DDDamiennn,39 ans.\nEsthète et réservé.\nProfesseur d'Ita...,238381,930,3615,331185,574111
2228,Le360fr,Le360,Le média du #Maroc 🇲🇦 \nle360.ma | AR @le360a...,145935,3,426494,163,572595
2886,ONU_fr,Nations Unies (ONU),"Compte officiel des Nations Unies. \nPaix, dig...",35050,1374,532908,1457,570789


In [87]:
import csv

dict_acteurs_final_desc.to_csv(
    'PPD_ALL_Scrapping_acteurs.csv',
    sep=';',
    index=False,
    encoding='utf-8-sig',
    #quoting=csv.QUOTE_NONE,
    #escapechar='\\'
)

In [88]:
dict_acteurs_final_desc.to_excel(r'PPD_ALL_Scrapping_acteurs.xlsx', index=False)